In [1]:
import pandas as pd
import numpy as np
import env
import os
from sklearn.model_selection import train_test_split
from wrangle_zillow import acquire_zillow, handle_missing_values, null_table
import warnings
warnings.filterwarnings("ignore")

# Exercises

## Acquire and Summarize

### Acquire data from the cloud database.
You will want to end with a single dataframe. Include the logerror field and all other fields related to the properties that are available. You will end up using all the tables in the database.

Be sure to do the correct join (inner, outer, etc.). We do not want to eliminate properties purely because they may have a null value for airconditioningtypeid. - Only include properties with a transaction in 2017, and include only the last transaction for each property (so no duplicate property ID's), along with zestimate error and date of transaction. (Hint: read the docs for the .duplicated method) - Only include properties that have a latitude and longitude value.

In [2]:
df = acquire_zillow()
df.head()

,id,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,bathroomcnt,...,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,propertylandusedesc,storydesc,typeconstructiondesc,logerror,parcelid
0,0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,3.5,...,6.059063e+13,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,0.025595,14297519
1,1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,1.0,...,6.111001e+13,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,0.055619,17052889
2,2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,2.0,...,6.059022e+13,NaN,NaN,NaN,NaN,Single Family Residential,NaN,NaN,0.005383,14186244
3,3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,NaN,3.0,...,6.037300e+13,NaN,NaN,NaN,Central,Single Family Residential,NaN,NaN,-0.103410,12177905
4,4,NaN,NaN,266.0,2.0,NaN,NaN,1.0,NaN,3.0,...,6.037124e+13,Central,NaN,NaN,Central,Condominium,NaN,NaN,0.006940,10887214


In [3]:
df['id'].value_counts()

id
0        1
51745    1
51743    1
51742    1
51741    1
        ..
25872    1
25871    1
25870    1
25869    1
77612    1
Name: count, Length: 77579, dtype: int64

In [4]:
df = df.drop(columns={'parcelid'})

### Summarize your data (summary stats, info, dtypes, shape, distributions, value_counts, etc.)

In [5]:
df.describe()

,id,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,bathroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyyear,censustractandblock,logerror
count,77579.000000,223.000000,50.0,77579.000000,49571.000000,15.000000,207.000000,25007.000000,50.000000,77579.000000,...,17599.000000,172.0,7.746400e+04,7.757800e+04,77579.0,7.757700e+04,77574.000000,2900.000000,7.733200e+04,77579.000000
mean,38806.223579,6.040359,7.0,261.824476,3.921749,3.933333,7.386473,1.812013,679.720000,2.298496,...,1.434286,1.0,1.892811e+05,4.901506e+05,2016.0,3.011515e+05,5995.961788,14.088276,6.049667e+13,0.016803
std,22403.467491,0.556035,0.0,5.141596,3.594780,0.258199,2.728030,2.965768,689.703546,0.996732,...,0.544515,0.0,2.304107e+05,6.537979e+05,0.0,4.927249e+05,7628.859728,2.181281,1.533339e+12,0.170738
min,0.000000,4.000000,7.0,31.000000,1.000000,3.000000,2.000000,1.000000,38.000000,0.000000,...,1.000000,1.0,4.400000e+01,1.000000e+03,2016.0,1.610000e+02,19.920000,3.000000,6.037101e+13,-4.655420
25%,19404.500000,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,273.000000,2.000000,...,1.000000,1.0,8.417925e+04,2.068985e+05,2016.0,8.529300e+04,2712.630000,14.000000,6.037311e+13,-0.024309
50%,38804.000000,6.000000,7.0,261.000000,2.000000,4.000000,7.000000,1.000000,515.000000,2.000000,...,1.000000,1.0,1.364045e+05,3.588785e+05,2016.0,2.031880e+05,4448.265000,15.000000,6.037603e+13,0.006676
75%,58207.500000,6.000000,7.0,266.000000,7.000000,4.000000,7.000000,1.000000,796.500000,3.000000,...,2.000000,1.0,2.187340e+05,5.690000e+05,2016.0,3.667530e+05,6926.917500,15.000000,6.059042e+13,0.039291
max,77612.000000,13.000000,7.0,275.000000,24.000000,4.000000,21.000000,13.000000,3560.000000,18.000000,...,6.000000,1.0,1.142179e+07,4.906124e+07,2016.0,4.895220e+07,586639.300000,99.000000,4.830301e+14,5.262999


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77579 entries, 0 to 77578
Data columns (total 66 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77579 non-null  int64  
 1   typeconstructiontypeid        223 non-null    float64
 2   storytypeid                   50 non-null     float64
 3   propertylandusetypeid         77579 non-null  float64
 4   heatingorsystemtypeid         49571 non-null  float64
 5   buildingclasstypeid           15 non-null     float64
 6   architecturalstyletypeid      207 non-null    float64
 7   airconditioningtypeid         25007 non-null  float64
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77579 non-null  float64
 10  bedroomcnt                    77579 non-null  float64
 11  buildingqualitytypeid         49809 non-null  float64
 12  calculatedbathnbr             76963 non-null  float64
 13  d

In [7]:
df.dtypes

id                          int64
typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
                           ...   
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
logerror                  float64
Length: 66, dtype: object

In [8]:
df.shape

(77579, 66)

In [9]:
df.value_counts()

Series([], Name: count, dtype: int64)

### Write a function that takes in a dataframe of observations and attributes and returns a dataframe where each row is an atttribute name, the first column is the number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute. Run the function and document takeaways from this on how you want to handle missing values.

The output should look like the table below:

	 				num_rows_missing		pct_rows_missing
	parcelid			0				0.000000
	airconditioningtypeid		29041				0.535486
	architecturalstyletypeid	54232				0.999982

In [10]:
def null_table(df):
    '''
    null_table will take in a dataframe and create a new dataframe showing total null values for each column and the percentage of null values to the total column size.
    '''
    null_df = pd.DataFrame(data=[{'column_name':df.columns, 'num_rows_missing':df.isnull().sum(), 'pct_rows_missing':round(((df.isnull().sum())/df.shape[0])* 100,2)}])
    k = 0
    for col in df.columns:
        null_df.loc[k] = [df.columns[k], df.isnull().sum()[k], round(((df.isnull().sum()[k])/df.shape[0])* 100,2)]
        k += 1
    return null_df

In [11]:
null_table(df)

,column_name,num_rows_missing,pct_rows_missing
0,id,0,0.0
1,typeconstructiontypeid,77356,99.71
2,storytypeid,77529,99.94
3,propertylandusetypeid,0,0.0
4,heatingorsystemtypeid,28008,36.1
...,...,...,...
61,heatingorsystemdesc,28068,36.18
62,propertylandusedesc,0,0.0
63,storydesc,77529,99.94
64,typeconstructiondesc,77356,99.71


#### There are definitely a ton of columns that need cleaning, we have a lot of null values that make up a large majority of the 66 existing columns.

## Prepare

### Remove any properties that are likely to be something other than single unit properties. (e.g. no duplexes, no land/lot, ...). There are multiple ways to estimate that a property is a single unit, and there is not a single "right" answer.

In [12]:
df['propertylandusedesc'].value_counts()

propertylandusedesc
Single Family Residential                     52441
Condominium                                   19342
Duplex (2 Units, Any Combination)              2021
Planned Unit Development                       1947
Quadruplex (4 Units, Any Combination)           729
Triplex (3 Units, Any Combination)              539
Cluster Home                                    335
Mobile Home                                      74
Manufactured, Modular, Prefabricated Homes       59
Residential General                              42
Cooperative                                      29
Commercial/Office/Residential Mixed Used         15
Townhouse                                         6
Name: count, dtype: int64

In [13]:
df['propertylandusedesc'] = (df['propertylandusedesc'] == ('Single Family Residential' or 'Mobile Home' or 'Manufactured, Modular, Prefabricated Homes'))
df = df.rename(columns={'propertylandusedesc':'single_unit'})
df = df.drop(df[df['single_unit'] == False].index)
df.head()

,id,typeconstructiontypeid,storytypeid,propertylandusetypeid,heatingorsystemtypeid,buildingclasstypeid,architecturalstyletypeid,airconditioningtypeid,basementsqft,bathroomcnt,...,taxdelinquencyyear,censustractandblock,airconditioningdesc,architecturalstyledesc,buildingclassdesc,heatingorsystemdesc,single_unit,storydesc,typeconstructiondesc,logerror
0,0,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,3.5,...,NaN,6.059063e+13,NaN,NaN,NaN,NaN,True,NaN,NaN,0.025595
1,1,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,6.111001e+13,NaN,NaN,NaN,NaN,True,NaN,NaN,0.055619
2,2,NaN,NaN,261.0,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,6.059022e+13,NaN,NaN,NaN,NaN,True,NaN,NaN,0.005383
3,3,NaN,NaN,261.0,2.0,NaN,NaN,NaN,NaN,3.0,...,NaN,6.037300e+13,NaN,NaN,NaN,Central,True,NaN,NaN,-0.103410
6,6,NaN,NaN,261.0,2.0,NaN,NaN,1.0,NaN,3.0,...,NaN,6.037461e+13,Central,NaN,NaN,Central,True,NaN,NaN,-0.001011


###  Create a function that will drop rows or columns based on the percent of values that are missing: handle_missing_values(df, prop_required_column, prop_required_row).
- The input:
  - A dataframe
  - A number between 0 and 1 that represents the proportion, for each column, of rows with non-missing values required to keep the column. i.e. if prop_required_column = .6, then you are requiring a column to have at least 60% of values not-NA (no more than 40% missing).
  - A number between 0 and 1 that represents the proportion, for each row, of columns/variables with non-missing values required to keep the row. For example, if prop_required_row = .75, then you are requiring a row to have at least 75% of variables with a non-missing value (no more that 25% missing).
- The output:
  - The dataframe with the columns and rows dropped as indicated. Be sure to drop the columns prior to the rows in your function.
- hint:
  - Look up the dropna documentation.
  - You will want to compute a threshold from your input values (prop_required) and total number of rows or columns.

In [14]:
df.iloc[4]

id                               6
typeconstructiontypeid         NaN
storytypeid                    NaN
propertylandusetypeid        261.0
heatingorsystemtypeid          2.0
                            ...   
heatingorsystemdesc        Central
single_unit                   True
storydesc                      NaN
typeconstructiondesc           NaN
logerror                 -0.001011
Name: 6, Length: 66, dtype: object

In [15]:
df.isnull().sum()[65]/df.shape[0] < .8

True

In [16]:
df.iloc[10].isnull().sum()/df.shape[1] < .5

False

First attempt

def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    handle_missing_values will take the dataframe, the threshold for our null values in our columns in the form of a float between 0 and 1, 
    and the threshold for our null values in our rows in the form of a float between 0 and 1
    return: cleaned dateframe with dropped null values above threshold desired.
    '''
    df = df
    k = 0
    for col in df.columns:
        if df.isnull().sum()[k]/df.shape[0] < prop_required_column:
            df = df.dropna()
            k += 1
        else: df = df
    k = 0
    for row in df.iloc[k]:
        if df.iloc[k].isnull().sum()/df.shape[1] < prop_required_row:
            df = df.dropna()
            k += 1
        else: df = df
    return df

secondary attempt with restructuring

def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    handle_missing_values will take the dataframe, the threshold for our null values in our columns in the form of a float between 0 and 1, 
    and the threshold for our null values in our rows in the form of a float between 0 and 1
    return: cleaned dateframe with dropped null values above threshold desired.
    '''
    df = df
    k = 0
    for col in df.columns:
        if df.isnull().sum()[k]/df.shape[0] < prop_required_column:
            df = df.dropna(axis = 1, thresh = prop_required_column)
        else: df = df
        k += 1
    k = 0
    for row in df.iterrows():
        if df.iloc[k].isnull().sum()/df.shape[1] < prop_required_row:
            df = df.dropna(axis = 0, thresh = prop_required_row)
        else: df = df
        k += 1
    return df

In [17]:
def handle_missing_values(df, prop_required_column, prop_required_row):
    '''
    handle_missing_values will take the dataframe, the threshold for our null values in our columns in the form of a float between 0 and 1,
    and the threshold for our null values in our rows in the form of a float between 0 and 1
    return: cleaned dateframe with dropped null values above threshold desired.
    '''
    # iterate through every column name in df:
    for col in df.columns:
        # check the ratio of missing values:
        if df[col].isnull().sum()/df.shape[0] > prop_required_column:
            # drop this specific column if its not up to snuff
            df = df.drop(columns=col)
    # for every index and row in the dataframe:
    for i, row in df.iterrows():
        # if the row null cells ratio do not meet what you want:
        if (row.isnull().sum()/df.shape[1]) > prop_required_row:
            # use the index i to drop that specific row
            df = df.drop(index=i)
    return df

In [18]:
df = handle_missing_values(df, 0.8, 0.75)

### Encapsulate your work inside of functions in a wrangle_zillow.py module.

### Done.

In [19]:
null_table(df)

,column_name,num_rows_missing,pct_rows_missing
0,id,0,0.0
1,propertylandusetypeid,0,0.0
2,heatingorsystemtypeid,18506,35.29
3,airconditioningtypeid,38803,73.99
4,bathroomcnt,0,0.0
5,bedroomcnt,0,0.0
6,buildingqualitytypeid,18701,35.66
7,calculatedbathnbr,137,0.26
8,calculatedfinishedsquarefeet,82,0.16
9,finishedsquarefeet12,247,0.47


In [20]:
handle_missing_values(df, .8, .7)

,id,propertylandusetypeid,heatingorsystemtypeid,airconditioningtypeid,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,airconditioningdesc,heatingorsystemdesc,single_unit,logerror
0,0,261.0,NaN,NaN,3.5,4.0,NaN,3.5,3100.0,3100.0,...,485713.0,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,NaN,NaN,True,0.025595
1,1,261.0,NaN,NaN,1.0,2.0,NaN,1.0,1465.0,1465.0,...,88000.0,464000.0,2016.0,376000.0,5672.48,6.111001e+13,NaN,NaN,True,0.055619
2,2,261.0,NaN,NaN,2.0,3.0,NaN,2.0,1243.0,1243.0,...,85289.0,564778.0,2016.0,479489.0,6488.30,6.059022e+13,NaN,NaN,True,0.005383
3,3,261.0,2.0,NaN,3.0,4.0,8.0,3.0,2376.0,2376.0,...,108918.0,145143.0,2016.0,36225.0,1777.51,6.037300e+13,NaN,Central,True,-0.103410
6,6,261.0,2.0,1.0,3.0,4.0,9.0,3.0,2962.0,2962.0,...,276684.0,773303.0,2016.0,496619.0,9516.26,6.037461e+13,Central,Central,True,-0.001011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77573,77607,261.0,2.0,NaN,2.0,4.0,6.0,2.0,1633.0,1633.0,...,125466.0,346534.0,2016.0,221068.0,4175.08,6.037555e+13,NaN,Central,True,0.001082
77575,77609,261.0,2.0,NaN,2.0,2.0,6.0,2.0,1286.0,1286.0,...,70917.0,354621.0,2016.0,283704.0,4478.43,6.037101e+13,NaN,Central,True,0.020615
77576,77610,261.0,NaN,NaN,2.0,4.0,NaN,2.0,1612.0,1612.0,...,50683.0,67205.0,2016.0,16522.0,1107.48,6.111008e+13,NaN,NaN,True,0.013209
77577,77611,261.0,2.0,1.0,1.0,3.0,4.0,1.0,1032.0,1032.0,...,32797.0,49546.0,2016.0,16749.0,876.43,6.037434e+13,Central,Central,True,0.037129
